In [2]:
from node2vec import Node2Vec
import os
import networkx as nx
import numpy as np
import random

In [3]:
#load genes for making the graph
geneidx = [] 
filea = open("data/gidx.txt")
node_gene = filea.readlines()
for id in node_gene:
    node = id.split()[0]
    geneidx.append(node)

In [4]:
#load diseases ids
doidx = []
filea = open("data/didx.txt")
node_disease = filea.readlines()
for id in node_disease:
    node_1 = id.split()[0]
    doidx.append(node_1)

In [5]:
#load lnRNA ids
lncidx = []
with open("data/lncRNA_ids.txt", 'r') as file:
    node_lncRNA = file.readlines()
    for id in node_lncRNA:
        node = id.split()[0]
        lncidx.append(node)

In [6]:
#load miRNA
miidx = []
with open('data/miRNA_ids.txt', 'r') as file:
    node_miRNA = file.readlines()
    for id in node_miRNA:
        node = id.split()[0]
        miidx.append(node)

In [7]:
filea = open("data/X_train.txt")
x_train = filea.readlines()
# print(x_train[0].split()[0].split(",")[0])
fileb = open("data/y_train.txt")
y_train = fileb.readlines()

filea = open("data/X_test.txt")
X_test = filea.readlines()
fileb = open("data/y_test.txt")
y_test = fileb.readlines()

In [8]:
# train set
train = []
for i in range(len(x_train)):
        tup1 = (x_train[i].split()[0].split(",")[0],) #G123
        tup2 = (x_train[i].split()[0].split(",")[1],) #D123
        tup3 = (y_train[i].split()[0],)               #1/0
        tup4 = tup1 + tup2 + tup3
        # print(tup4)
        train.append(tup4)
print(train[0])

('G205', 'D47', '0')


In [9]:
# test set      
test = []
for i in range(len(X_test)):
        tup1 = (X_test[i].split()[0].split(",")[0],)
        tup2 = (X_test[i].split()[0].split(",")[1],)
        tup3 = (y_test[i].split()[0],)
        tup4 = tup1 + tup2 + tup3
        test.append(tup4) 

In [10]:
#embedding the 3 graphs: gd, gld, gmd
graph_name = 'gda'

gda_graph = nx.read_weighted_edgelist('data/'+graph_name+'.edgelist',delimiter="\t")  
print(gda_graph)

print("Before:",gda_graph.number_of_nodes(),gda_graph.number_of_edges())
for edge in list(gda_graph.edges):
    if gda_graph.get_edge_data(edge[0],edge[1])['weight']<=0.0:
        gda_graph.remove_edge(edge[0],edge[1])
print("After:",gda_graph.number_of_nodes(),gda_graph.number_of_edges())

node2vec = Node2Vec(gda_graph, dimensions=64, walk_length=10, num_walks=100, workers=4) 
model = node2vec.fit(window=10, min_count=1, batch_words=4)
print("finished!")

Graph with 1217 nodes and 217000 edges
Before: 1217 217000
After: 1217 5696


Computing transition probabilities:   0%|          | 0/1217 [00:00<?, ?it/s]

finished!


In [11]:
#word vectors learnt by node2vec, according to the graph
gd_g_vec={} 
for gid in geneidx:
    gd_g_vec[gid]=model.wv.get_vector(gid)  
    
gd_d_vec={}                 
for dis in doidx:
    gd_d_vec[dis]=model.wv.get_vector(dis)  

# gd_l_vec={}
# for lnc in lncidx:
#     gd_l_vec[lnc] = model.wv.get_vector(lnc)

# gd_m_vec = {}
# for mi in miidx:
#     gd_m_vec[mi] = model.wv.get_vector(mi)

In [28]:
X_train_gd =[]          
for gd in train:                    
    gd_vec = np.concatenate((gd_g_vec[gd[0]],gd_d_vec[gd[1]])) 
    X_train_gd.append(gd_vec)
        
X_train_gd = np.array(X_train_gd)

X_test_gd =[]
for gd in test:
    gd_vec = np.concatenate((gd_g_vec[gd[0]],gd_d_vec[gd[1]]))
    X_test_gd.append(gd_vec)

X_test_gd = np.array(X_test_gd) 
print(X_train_gd.shape, X_test_gd.shape)

(9112, 128) (2280, 128)


In [29]:
np.save('data/X_train_gd_node2vec.npy',X_train_gd) 
np.save('data/X_test_gd_node2vec.npy',X_test_gd)

print("save successful！！！")

save successful！！！


In [ ]:
#hif1a is gene 1; use the gene name.csv for the gene ids, not the genex.csv